import sys
!{sys.executable} -m pip install pandas==1.5.2

In [1]:

#import sys
#!{sys.executable} -m pip install html5lib

In [2]:
# import modules/libraries
import warnings 
warnings.simplefilter(action='ignore')
from entsoe import EntsoePandasClient
import pandas as pd
import json
import requests
from pandas import json_normalize

import osmnx as ox
import pandas as pd
import numpy as np
import geopandas as gpd
import time
from datetime import datetime
from scipy import stats
import itertools
import os
import pickle
import geojson
from sqlalchemy import create_engine
import re
import sqlite3
from pathlib import Path
from shapely.geometry.polygon import Polygon
from shapely.geometry.multipolygon import MultiPolygon
import chardet
from scipy import spatial
from scipy.spatial import KDTree
from shapely import wkt

cwd = Path().resolve()

# visualisation
import plotly.io as pio
import plotly.graph_objects as go
import plotly.express as px
import plotly.figure_factory as ff
import seaborn as sns
import matplotlib as mpl 
%matplotlib inline 
import matplotlib.pyplot as plt

pd.set_option('display.max_columns', None)

In [3]:
client = EntsoePandasClient(api_key="2d80d663-6eb6-447f-b31b-9d5c6b41365a")


https://github.com/EnergieID/entsoe-py/blob/master/entsoe/mappings.py

country codes and examples:
https://thesmartinsights.com/how-to-query-data-from-the-entso-e-transparency-platform-using-python/


In [ ]:
start = pd.Timestamp('20221209', tz='Europe/Brussels')
end = pd.Timestamp('20221210', tz='Europe/Brussels')
country_code = 'AT'  # Belgium
country_code_from = 'FR'  # France
country_code_to = 'DE_LU' # Germany-Luxembourg
type_marketagreement_type = 'A01'
contract_marketagreement_type = 'A01'

data = {}


eu_list = ['AT', 'BE', 'BG','HR', 'CZ', 'DE_LU', 'DK', 'EE', 'FI', 'FR', 'GR', 'HU', 'IE', 'IT', 'LV',
          'NL']
for country in eu_list:
    try:
        s = client.query_day_ahead_prices(country, start=start,end=end)
        df = s.to_frame()
        df.rename(columns={df.columns[0]: 'price_mwh'},inplace=True)
        mean_price = df.price_mwh.mean()
        print(mean_price, country)
        #df.head(24)
        data[country] = mean_price
    except:
        print("problem in ", country)
    

Connection Error, retrying in 0 seconds


problem in  AT


Connection Error, retrying in 0 seconds


problem in  BE


Connection Error, retrying in 0 seconds


...

problem in  CZ


Connection Error, retrying in 0 seconds


problem in  DE_LU


Connection Error, retrying in 0 seconds


problem in  DK
404.0900000000001 EE


In [ ]:
data

In [ ]:
s = client.query_day_ahead_prices('IT_NORD', start=start,end=end)
df = s.to_frame()
df.rename(columns={df.columns[0]: 'price_mwh'},inplace=True)
mean_price = df.price_mwh.mean()
print(mean_price)
df.head(24)

In [ ]:
start = pd.Timestamp('20221204', tz='Europe/Brussels')
end = pd.Timestamp('20221205', tz='Europe/Brussels')
t =client.query_generation_per_plant(country_code, start=start,end=end, psr_type=None)
t.head()

### Windparks Österreich

https://globalenergymonitor.org/projects/global-wind-power-tracker/tracker-map/
https://windeurope.org/intelligence-platform/product/european-offshore-wind-farms-map-public/
https://structurae.net/en/structures/towers/wind-turbines/list

In [ ]:
def get_list_austria():
    """  """
    r = requests.get('https://www.thewindpower.net/country_windfarms_en_13_austria.php')
    # print(r.content)

    df = pd.read_html('https://www.thewindpower.net/country_windfarms_en_13_austria.php', header=0, displayed_only=False)[3]
    # df = pd.read_html('https://www.thewindpower.net/country_windfarms_en_13_austria.php', skiprows = 0, header=0, extract_links='all')[3]
    
    df=df.dropna(subset=['Name'])
    return df

df = get_list_austria()
print(df.shape[0])
#df.tail(10)

In [ ]:
# https://www.igwindkraft.at/maps/current/
r = requests.get('https://www.igwindkraft.at/src_project/external/maps/generated/gmaps_daten.js')
#print(r.encoding)
d = json.loads(str(r.content).split('= ')[1].split("\'")[0])

In [ ]:
df_tmp = pd.DataFrame.from_dict(d[0]['places'])
split_df = pd.DataFrame(df_tmp['data'].tolist())
split_df.rename(columns={0:"ort", 1:"betreiber", 3:"windräder", 4:"leistung", 5:"type", 6:"jahr", 8:"lat", 9:"lon", 11:"turbine", 12:"nabenhöhe", 13:"durchmesser"}, inplace=True)
split_df.head(1)

In [ ]:
import plotly.express as px
import plotly.graph_objects as go


fig = go.Figure(go.Scattermapbox(mode='markers',lat=split_df.lat.tolist(), lon=split_df.lon.tolist(), marker={'size': 10}, 
                                # hoverinfo='skip',
                                 text = split_df.ort.tolist(),
                              # hover_name="ort",
                                 showlegend=False))
#add_trace(fig, parameters[1], symbol="cafe", name="cafe", size=4)
fig.update_layout(mapbox_style="light", mapbox_accesstoken="pk.eyJ1IjoiYXJlYnVyZyIsImEiOiJjbDZkZzdhOXAwNTk5M2ZteDdkY3lvd2FsIn0.c4hsFv69HozIVHq7ItrTzg", mapbox_zoom=6,
                 mapbox_center={"lat": 48.210033, "lon": 16.363449})
fig.update_layout(font=dict(family="Helvetica"), legend={"title": "Select category"})
fig.update_layout(margin={"r": 0, "t": 0, "l": 0, "b": 0}, paper_bgcolor='rgba(0,0,0,0)',
              plot_bgcolor='rgba(0,0,0,0)')
fig.show()

In [ ]:
# attrs = {'class': 'wikitable sortable toptextcells zebra jquery-tablesorter'}
kraftwerke_verbund = pd.read_html('https://de.wikipedia.org/wiki/Liste_von_Kraftwerken_der_Verbund_AG', skiprows = 0, header=0)[0]
kraftwerke_verbund.tail()



In [ ]:
t= kraftwerke_verbund.copy()

def get_lat_long(name, land_raw):
    """ get latitude and longitude coordinate from POINT geometry """
    if land_raw == 'Stmk':
        bundesland = "Steiermark"
    elif land_raw == 'Sbg':
        bundesland = "Salzburg"
    elif land_raw == 'OÖ':
        bundesland = "Oberösterreich"
    elif land_raw == 'NÖ':
        bundesland = "Niederösterreich"
    else:
        bundesland = land_raw
    text = f"{name}, {bundesland}"
    try:
        #return pd.Series([point.x, point.y])
        data_json = requests.get(url=f'https://nominatim.openstreetmap.org/search?q={text}&format=json&polygon=1&addressdetails=1').json()
        lat = data_json[0]['lat']
        lon = data_json[0]['lon']
        print(text, lat, lon)
        return pd.Series([lat, lon])
            
    except:
        print(text)
        return pd.Series([np.nan, np.nan])
        


t[['lat', 'lon']] = t.apply(lambda x: get_lat_long(x['Name'], x['Bundesland']), axis=1)

In [ ]:
t[t['Bundesland'] == "Sbg"].head()


In [ ]:
import plotly.express as px
import plotly.graph_objects as go

#data = pd.read_excel('tmp.xlsx', index_col=0,
#              na_values=['string1', 'string2'])  
#data = pd.read_csv('kraftwerke_salzburg.csv', sep=",", index_col=False)  
sbg_verbund = t[t['Bundesland'] == "Sbg"]

def kraftwerke_salzburg():
    fig = go.Figure(go.Scattermapbox(mode='markers',lat=sbg_verbund.lat.tolist(), lon=sbg_verbund.lon.tolist(), marker={'size': 10}, 
                                    # hoverinfo='skip',
                                     text = sbg_verbund.Name.tolist(),
                                  # hover_name="ort",
                                     showlegend=False))
    #add_trace(fig, parameters[1], symbol="cafe", name="cafe", size=4)
    fig.update_layout(mapbox_style="light", mapbox_accesstoken="pk.eyJ1IjoiYXJlYnVyZyIsImEiOiJjbDZkZzdhOXAwNTk5M2ZteDdkY3lvd2FsIn0.c4hsFv69HozIVHq7ItrTzg", mapbox_zoom=6,
                     mapbox_center={"lat": 48.210033, "lon": 16.363449})
    fig.update_layout(font=dict(family="Helvetica"), legend={"title": "Select category"})
    fig.update_layout(margin={"r": 0, "t": 0, "l": 0, "b": 0}, paper_bgcolor='rgba(0,0,0,0)',
                  plot_bgcolor='rgba(0,0,0,0)')
    fig.show()
kraftwerke_salzburg()

In [ ]:
kraftwerke_salzburg()

### Plot commodities

Documentation: https://www.commodities-api.com/documentation#historicalrates

Get the lastest data with commodieties API.

In [ ]:
class ApiError(Exception):
    pass

start_date = "2021-05-01"
end_date = "2022-05-25"

base_currency = 'TGZ22'
symbol = 'EUR'
endpoint = 'latest'
access_API = 'ns8db94qwty8z0d3qtjd831aytoys40jnnz2e814c0hwft056uy0sgfjwyfi'

#urls = f"https://www.commodities-api.com/api/{endpoint}?access_key={access_API}&base={base_currency}&symbols={symbol}"
#resp = requests.get(urls)

#start_date = "2022-11-08"
#url = f"https://www.commodities-api.com/api/open-high-low-close/{start_date}?access_key={access_API}&base=TGZ22&symbols=EUR"
# time series
resp = requests.get(f"https://www.commodities-api.com/api/timeseries?access_key={access_API}&start_date={start_date}&end_date={end_date}&base={base_currency}\
&symbols={symbol}")
if resp.status_code != 200:
    raise ApiError(f"Can't read OLHC values starting from {start_date}")
print(resp.text)
#print(url)

#### Long time series
https://tradingeconomics.com/commodity/eu-natural-gas

future prices: https://www.barchart.com/futures/quotes/TG*0/futures-prices


https://www.barchart.com/futures/quotes/TG*1#google_vignette


In [ ]:
s = requests.Session()

def getCookies(cookie_jar, domain):
    cookie_dict = cookie_jar.get_dict(domain=domain)
    found = ['%s=%s' % (name, value) for (name, value) in cookie_dict.items()]
    return ';'.join(found)

my_headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:75.0) Gecko/20100101 Firefox/75.0'}
s.headers.update(my_headers)

r = s.get("https://stackoverflow.com")
print(r.headers)
#print(r.content)
getCookies(r.cookies, ".stackoverflow.com")

In [ ]:
s = requests.Session()

def getCookies(cookie_jar, domain):
    cookie_dict = cookie_jar.get_dict(domain=domain)
    found = ['%s=%s' % (name, value) for (name, value) in cookie_dict.items()]
    return ';'.join(found)

# "set-cookie" : "TEServer=TEIIS; expires=Tue, 13-Dec-2022 09:25:29 GMT; path=/"
my_headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:75.0) Gecko/20100101 Firefox/75.0'}
s.headers.update(my_headers)
r = s.get("https://tradingeconomics.com/commodity/eu-natural-gas")
print(r.headers)
print(r.cookies)
#print(r.headers)
getCookies(r.cookies, ".tradingeconomics.com")

# 2.
print("------------------")
r = s.get("https://tradingeconomics.com/commodity/eu-natural-gas")
print(r.headers)
print(r.cookies)
print(r.content)


#cookies = {'enwiki_session': '17ab96bd8ffbe8ca58a78657a918558'}

#r = requests.post('http://wikipedia.org', cookies=cookies)

In [ ]:
s = requests.Session()
#data = {"login":"my_login", "password":"my_password"}
url = "https://tradingeconomics.com/commodity/eu-natural-gas"
r = s.post(url, data=data)
r.cookies




In [ ]:
url = "https://tradingeconomics.com/commodity/eu-natural-gas"
resp = requests.get("https://tradingeconomics.com/")
resp = requests.post(url, cookies=r1.cookies)
print(resp.text)
print(resp.status_code)

In [ ]:
def get_commodieties(name):
    if name == 'ttf':
        url=f"https://markets.tradingeconomics.com/chart?s=ngeu:com&interval=1d&span=5y&securify=new&url=/commodity/eu-natural-gas&AUTH=S%2F5%2B8%2BV0tSEqVO3f3fc8i7D1ALNK3vv44AAM1sLXABiksGpLT3z6hHgIC4khJSb1&ohlc=1"
    elif name == 'brent':
        url = "https://markets.tradingeconomics.com/chart?s=co1:com&span=5y&securify=new&url=/commodity/eu-natural-gas&AUTH=S%2F5%2B8%2BV0tSEqVO3f3fc8i7D1ALNK3vv44AAM1sLXABiksGpLT3z6hHgIC4khJSb1&ohlc=1"
    elif name == 'usdrub':
        url = "https://markets.tradingeconomics.com/chart?s=usdrub:cur&interval=1d&span=5y&securify=new&url=/russia/currency&AUTH=ZunHTz66JfXqLQTeiv78irx2z508SOi1CsDlRXaVUGUxcEGnev2LDgoaqNeHISd9&ohlc=1"
    elif name == "nbp":
        url = "https://markets.tradingeconomics.com/chart?s=nguk:com&interval=1d&span=5y&securify=new&url=/russia/currency&AUTH=ZunHTz66JfXqLQTeiv78irx2z508SOi1CsDlRXaVUGUxcEGnev2LDgoaqNeHISd9&ohlc=1"
    elif name == "CO2":
        url = "https://markets.tradingeconomics.com/chart?s=eecxm:ind&interval=1d&span=5y&securify=new&url=/commodity/carbon&AUTH=GPVSlEsoJTkYP45bnbNF28FlnikmX%2BwnW7RXI%2FnXTNxH1U1lhv9220niYGHm%2Fnda&ohlc=1"
    resp = requests.get(url)
    if resp.status_code != 200:
        raise ApiError(f"Problem")
    # print(resp.text)
    #t = str(resp.text)
    json_object = json.dumps(resp.text, indent=4)
    with open(f"{name}.json", "w") as outfile:
        outfile.write(resp.text)

#get_commodieties('ttf')
#get_commodieties('brent')
#get_commodieties('usdrub')
#get_commodieties('nbp')
get_commodieties('CO2')

In [ ]:
def get_historic_dates(filename="history.json"):
    try:
        f = open(f'{filename}')
        data = json.load(f)
        f.close()
        df_history = json_normalize(data)
        df_history['date'] = pd.to_datetime(df_history['date'], infer_datetime_format=True)
        return df_history
    except Exception as e:
        print(f"Error with json file syntax.")

In [ ]:
def get_value_at_day(df, date):
    val = df[df['date'] == date]['y'].iloc[0]
    return val


def improve_text_position(x):
    """ it is more efficient if the x values are sorted """
    positions = ['top center', 'bottom center']  # you can add more: left center ...
    return [positions[i % len(positions)] for i in range(len(x))]


def add_annotation(fig, df):
    df_history = get_historic_dates()
    for index, row in df_history.iterrows():
        if int(row['priority']) < 2:
            try:
                y = get_value_at_day(df, row['date'])
                fig.add_annotation(x=row['date'], y=y,
                    text=row['text'], showarrow=True, yshift=10, font=dict(family="Courier New, monospace", size=8, color="#636363"),
                    align="center", arrowhead=2, arrowsize=1, arrowwidth=2, arrowcolor="#636363",)
            except (IndexError) as e:
                print(f"At least one date is not yet in time series.")
                pass
    
            # fig.update_traces(textposition=improve_text_position(df['x']))
    return fig


def xx(fig, df, date, text):
    date_time_obj = datetime.strptime(date, '%Y-%m-%d')
    y = get_value_at_day(df, date)
    fig.add_annotation(x=date_time_obj, y=y,
        text=text, showarrow=True, yshift=10, font=dict(family="Courier New, monospace", size=10, color="#636363"),
        align="center", arrowhead=2, arrowsize=1, arrowwidth=2, arrowcolor="#636363",)
    return fig


def load_historic_chart_data(filename):
    f = open(f'{filename}.json')
    data = json.load(f)
    f.close()
    data = data['series'][0]['data']
    #print(data[0])

    df = json_normalize(data)
    df['date'] = pd.to_datetime(df['date'], infer_datetime_format=True) #, format='%Y-%m%d:%H:%M:%S.%f')

    df = df.iloc[-400:-1, 0:-1]
    return df


def candlestick_chart(filename="ttf", y_axis_title="", title=""):
    df = load_historic_chart_data(filename)
    fig = go.Figure(data=[go.Candlestick(x=df["date"], open=df["open"], high=df["high"],
                    low=df["low"], close=df["close"])])
    fig.update_layout(title=title)
        
    #add_annotation(fig, df, '2022-09-26', "Nordstream 2 <br>Explosion")
    #add_annotation(fig, df, '2022-02-24', "Russian invasion <br>of Ukraine")
    add_annotation(fig, df)
    fig.update(layout_xaxis_rangeslider_visible=False)
    
    
    fig.update_yaxes(tickfont=dict(family='Helvetica', size=12, color='#9c9c9c'), titlefont=dict(size=12), title_font_color='#9c9c9c', title_text=y_axis_title, mirror=True,
    ticks='outside', showline=True, gridwidth=1, gridcolor='#4c4c4c')
    fig.update_xaxes(tickfont=dict(family='Helvetica', size=12, color='#9c9c9c'), titlefont=dict(size=12), title_font_color='#9c9c9c', title_text="", mirror=True,
    ticks='outside', showline=True, gridwidth=1, gridcolor='#4c4c4c')
    fig.update_layout(font=dict(family="Helvetica"), legend_font_size=14)

    fig.update_layout(paper_bgcolor='rgba(0,0,0,0)', plot_bgcolor='rgba(0,0,0,0)')
    #fig.update_layout(margin={"r":0,"t":40,"l":0,"b":0})
    fig.show()


In [ ]:

#EU ETS Allowances
candlestick_chart(filename="CO2", y_axis_title="Eurot CO2 (CO2e)", title="EU Carbon Permits<br><sub>CO2 European Emission Allowances</sub>")

In [ ]:
candlestick_chart(filename="ttf", y_axis_title="spot price in €/MWh", title="Natural Gas EU<br><sub>Dutch TTF</sub>")

In [ ]:
candlestick_chart(filename="nbp", y_axis_title="spot price in GBp/Thm", title="Natural Gas UK<br><sub>NBP</sub>")

In [ ]:
candlestick_chart(filename="brent", y_axis_title="USD per barrel", title="Crude Oil<br><sub>dsdfsdf</sub>")

In [ ]:
candlestick_chart(filename="usdrub", title="USD/RUB")

In [ ]:
# candlestick_chart(filename="henry_hub", title="Henry Hub Natural Gas<br> USD per million BTU")

### List of all LNG Tankers

#### Sources:
https://gtt.fr/references-partners/built-vessels

http://www.aukevisser.nl/supertankers/gas-SP/id703.htm

https://www.vesselfinder.com/de/vessels/details/9288708

https://digital-geography.com/vessel-tracking-the-python-way/

https://pypi.org/project/Marine-Traffic-API/
https://www.marinetraffic.com/en/ais/home/centerx:3.8/centery:39.8/zoom:6

https://medium.com/analytics-vidhya/tracking-ships-and-visualize-them-in-qgis-35c074810937
